# Train Model with DPO

Code authored by: Shaw Talebi

https://github.com/ShawhinT/YouTube-Blog/tree/main/LLMs/dpo

### imports

In [1]:
from datasets import load_dataset
from trl import DPOConfig, DPOTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

### load data

In [2]:
dataset = load_dataset("shawhin/youtube-titles-dpo")

### load model

In [3]:
model_name = "Qwen/Qwen2.5-0.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token # set pad token

### generate title with base model

In [4]:
def format_chat_prompt(user_input, system_message="You are a helpful assistant."):
    """
    Formats user input into the chat template format with <|im_start|> and <|im_end|> tags.

    Args:
        user_input (str): The input text from the user.

    Returns:
        str: Formatted prompt for the model.
    """
    
    # Format user message
    user_prompt = f"<|im_start|>user\n{user_input}<|im_end|>\n"
    
    # Start assistant's turn
    assistant_prompt = "<|im_start|>assistant\n"
    
    # Combine prompts
    formatted_prompt = user_prompt + assistant_prompt
    
    return formatted_prompt

In [5]:
# Set up text generation pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device='auto')

# Example prompt
prompt = format_chat_prompt(dataset['valid']['prompt'][0][0]['content'])

# Generate output
outputs = generator(prompt, max_length=100, truncation=True, num_return_sequences=1, temperature=0.7)

print(outputs[0]['generated_text'])

Device set to use mps


<|im_start|>user
Given the YouTube video idea write an engaging title.

**Video Idea**: intro independent component analysis

**Additional Guidance**:
- Title should be between 30 and 75 characters long
- Only return the title idea, nothing else!<|im_end|>
<|im_start|>assistant
"Unlocking Independent Component Analysis: The Key to Understanding Your Data!"


### train model

In [6]:
ft_model_name = model_name.split('/')[1].replace("Instruct", "DPO")

training_args = DPOConfig(
    output_dir=ft_model_name, 
    logging_steps=25,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_strategy="epoch",
    eval_strategy="epoch",
    eval_steps=1,
)

device = torch.device('mps')

In [7]:
trainer = DPOTrainer(
    model=model, 
    args=training_args, 
    processing_class=tokenizer, 
    train_dataset=dataset['train'],
    eval_dataset=dataset['valid'],
)
trainer.train()

Epoch,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/chosen,Logps/rejected,Logits/chosen,Logits/rejected
1,0.560100,0.562754,2.503184,1.936778,0.658333,0.566406,-31.019386,-40.121445,-3.392133,-3.383956
2,0.408300,0.520149,1.080866,-0.061419,0.766667,1.142285,-45.242565,-60.103413,-3.431897,-3.411629
3,0.286900,0.582148,0.224948,-1.232885,0.725000,1.457833,-53.801743,-71.818069,-3.231009,-3.202194


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=387, training_loss=0.4348428033735093, metrics={'train_runtime': 471.6028, 'train_samples_per_second': 6.527, 'train_steps_per_second': 0.821, 'total_flos': 0.0, 'train_loss': 0.4348428033735093, 'epoch': 3.0})

### use fine-tuned model

In [8]:
# Load the fine-tuned model
ft_model = trainer.model

In [9]:
# Set up text generation pipeline
generator = pipeline("text-generation", model=ft_model, tokenizer=tokenizer, device='mps')

# Example prompt
prompt = format_chat_prompt(dataset['valid']['prompt'][0][0]['content'])

# Generate output
outputs = generator(prompt, max_length=100, truncation=True, num_return_sequences=1, temperature=0.7)

print(outputs[0]['generated_text'])

Device set to use mps


<|im_start|>user
Given the YouTube video idea write an engaging title.

**Video Idea**: intro independent component analysis

**Additional Guidance**:
- Title should be between 30 and 75 characters long
- Only return the title idea, nothing else!<|im_end|>
<|im_start|>assistant
Independent Component Analysis for Beginners


### push to HF hub

In [10]:
model_id = f"shawhin/{ft_model_name}"
trainer.push_to_hub(model_id)

training_args.bin:   0%|          | 0.00/6.20k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/shawhin/Qwen2.5-0.5B-DPO/commit/4f36e42b1a38ef4504cd2cde6f9799ad0ef8a36d', commit_message='shawhin/Qwen2.5-0.5B-DPO', commit_description='', oid='4f36e42b1a38ef4504cd2cde6f9799ad0ef8a36d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/shawhin/Qwen2.5-0.5B-DPO', endpoint='https://huggingface.co', repo_type='model', repo_id='shawhin/Qwen2.5-0.5B-DPO'), pr_revision=None, pr_num=None)

In [11]:
format_chat_prompt(dataset['valid']['prompt'][0][0]['content'])

'<|im_start|>user\nGiven the YouTube video idea write an engaging title.\n\n**Video Idea**: intro independent component analysis\n\n**Additional Guidance**:\n- Title should be between 30 and 75 characters long\n- Only return the title idea, nothing else!<|im_end|>\n<|im_start|>assistant\n'